# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.drop(cities_df.columns[[0]], axis=1, inplace=True)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ongandjera,-17.88,15.07,78.78,34,65,7.96,NaN,1666473447
1,port alfred,-33.59,26.89,64.18,81,24,5.99,ZA,1666473447
2,waipawa,-41.41,175.52,56.48,54,1,6.11,NZ,1666473448
3,cabo san lucas,22.89,-109.91,90.61,70,97,11.97,MX,1666473448
4,benton harbor,42.12,-86.45,79.45,41,0,12.66,US,1666473391
...,...,...,...,...,...,...,...,...,...
568,san felipe,10.34,-68.74,79.38,86,49,6.53,VE,1666473660
569,xuzhou,34.18,117.16,56.08,53,37,7.31,CN,1666473660
570,manokwari,-0.87,134.08,78.69,79,70,3.36,ID,1666473661
571,lerwick,60.15,-1.15,51.80,93,87,16.58,GB,1666473441


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity_level = cities_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_level, 
                                 dissipating=False,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities_nice_weather = cities_df[(cities_df['Max Temp'] >= 60) &
                                (cities_df['Max Temp'] <= 80) &
                                (cities_df['Wind Speed'] <= 3) &
                                (cities_df['Cloudiness'] <= 10)].dropna().reset_index(drop=True)

hotel_df = cities_nice_weather.loc[:, ['City', 'Country']]
hotel_df['Hotel Name'] = ""
hotel_df['Lat'] = ""
hotel_df['Lng'] = ""
hotel_df

,City,Country,Hotel Name,Lat,Lng
0,taft,IR,,,
1,santa maria,BR,,,
2,cape town,ZA,,,
3,kazerun,IR,,,
4,bantou,CN,,,
5,algiers,DZ,,,
6,pierre,US,,,
7,nantucket,US,,,
8,constantine,DZ,,,
9,turayf,SA,,,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "type": "lodging",
    "key": g_key
}

for index, row in cities_nice_weather.iterrows():

    params['location'] = str(row['Lat']) + "," + str(row['Lng'])

    print(f"Retrieving Result {index} for {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = float(results[0]['geometry']['location']['lat'])
        hotel_df.loc[index, 'Lng'] = float(results[0]['geometry']['location']['lng'])
        
    except (IndexError, KeyError):
        print("Missing field/result... deleting city.")
        hotel_df = hotel_df.drop(index)
        
    print("------------")

Retrieving Result 0 for taft.
------------
Retrieving Result 1 for santa maria.
------------
Retrieving Result 2 for cape town.
------------
Retrieving Result 3 for kazerun.
------------
Retrieving Result 4 for bantou.
------------
Retrieving Result 5 for algiers.
------------
Retrieving Result 6 for pierre.
------------
Retrieving Result 7 for nantucket.
------------
Retrieving Result 8 for constantine.
------------
Retrieving Result 9 for turayf.
------------


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))